# Notebook 116: オプティマイザの選び方と実践ガイド

## Optimizer Selection Guide and Best Practices

---

### このノートブックの位置づけ

**Phase 10「最適化手法」** の最終章として、これまで学んだ内容を統合し、実践的なオプティマイザ選択ガイドを提供します。

### 学習目標

1. タスクに応じた **オプティマイザの選択基準** を理解する
2. **ハイパーパラメータチューニング** のベストプラクティスを学ぶ
3. **デバッグと診断** の方法を習得する
4. **実践的なレシピ** を身につける

### 前提知識

- Notebook 110-115 の内容

---

## 目次

1. [オプティマイザ選択フローチャート](#1-オプティマイザ選択フローチャート)
2. [タスク別推奨設定](#2-タスク別推奨設定)
3. [ハイパーパラメータチューニング](#3-ハイパーパラメータチューニング)
4. [学習のデバッグと診断](#4-学習のデバッグと診断)
5. [実践的なレシピ集](#5-実践的なレシピ集)
6. [よくある問題と解決策](#6-よくある問題と解決策)
7. [Phase 10 総まとめ](#7-phase-10-総まとめ)

In [ ]:
# 環境セットアップ
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)

print("環境セットアップ完了")

---

## 1. オプティマイザ選択フローチャート

In [ ]:
print("="*80)
print("オプティマイザ選択フローチャート")
print("="*80)
print("""
                              [START]
                                 |
                    ┌────────────┴────────────┐
                    │   どんなタスク？          │
                    └────────────┬────────────┘
          ┌──────────────────────┼──────────────────────┐
          │                      │                      │
     [画像分類]             [NLP/Transformer]      [その他/不明]
          │                      │                      │
     ┌────┴────┐            ┌────┴────┐                 │
     │バッチは？│            │バッチは？│                 │
     └────┬────┘            └────┬────┘                 │
    ┌─────┴─────┐          ┌─────┴─────┐                │
  [小~中]    [大]        [小~中]     [大]               │
    │         │            │          │                 │
  SGD+       LARS        AdamW       LAMB               │
 Momentum                                               │
                                                        │
                                              ┌─────────┴─────────┐
                                              │  まずは Adam で   │
                                              │  試してみる       │
                                              └───────────────────┘

""")

print("【補足】")
print("- 小~中バッチ: 32-256")
print("- 大バッチ: 1024以上")
print("- 迷ったら Adam から始める")
print("- ファインチューニングでは AdamW + 小さな学習率")

---

## 2. タスク別推奨設定

In [ ]:
print("="*80)
print("タスク別推奨設定")
print("="*80)
print("")

print("【1. 画像分類（CNN）】")
print("-" * 60)
print("オプティマイザ: SGD + Momentum (0.9)")
print("学習率: 0.1")
print("Weight Decay: 1e-4")
print("スケジューラ: MultiStepLR (milestones=[30, 60, 90])")
print("バッチサイズ: 128-256")
print("エポック数: 90-200")
print("")

print("【2. 物体検出（YOLO, Faster R-CNN等）】")
print("-" * 60)
print("オプティマイザ: SGD + Momentum (0.9)")
print("学習率: 0.01")
print("Weight Decay: 5e-4")
print("スケジューラ: Warmup + Step/Cosine")
print("バッチサイズ: 16-64（メモリ制約）")
print("")

print("【3. NLP - Transformer（BERT等）】")
print("-" * 60)
print("オプティマイザ: AdamW")
print("学習率: 1e-4 ~ 5e-4")
print("β1=0.9, β2=0.999")
print("Weight Decay: 0.01")
print("スケジューラ: Linear Warmup + Linear Decay")
print("Warmup: 全ステップの10%")
print("")

print("【4. ファインチューニング】")
print("-" * 60)
print("オプティマイザ: AdamW")
print("学習率: 1e-5 ~ 3e-5（事前学習時の1/10程度）")
print("Weight Decay: 0.01")
print("スケジューラ: Warmup + Cosine/Linear Decay")
print("エポック数: 3-10")
print("")

print("【5. GAN】")
print("-" * 60)
print("オプティマイザ: Adam（G, D両方）")
print("学習率: 2e-4")
print("β1=0.5, β2=0.999（標準とは異なる）")
print("Weight Decay: 0")
print("バッチサイズ: 64-128")
print("")

print("【6. 強化学習（PPO等）】")
print("-" * 60)
print("オプティマイザ: Adam")
print("学習率: 3e-4")
print("ε: 1e-5 (より大きい値)")
print("Gradient Clipping: max_norm=0.5")

---

## 3. ハイパーパラメータチューニング

### 3.1 学習率の探索

In [ ]:
def lr_range_test(loss_fn, grad_fn, start, start_lr=1e-7, end_lr=10, num_iter=100):
    """
    Learning Rate Range Test
    
    学習率を指数的に増加させながら損失を記録
    """
    pos = np.array(start, dtype=float)
    
    # 学習率の系列
    mult = (end_lr / start_lr) ** (1 / num_iter)
    lrs = [start_lr * (mult ** i) for i in range(num_iter)]
    
    losses = []
    smooth_loss = 0
    beta = 0.98
    
    for i, lr in enumerate(lrs):
        # 損失と勾配を計算
        loss = loss_fn(pos[0], pos[1])
        grad = grad_fn(pos[0], pos[1])
        
        # 指数移動平均
        smooth_loss = beta * smooth_loss + (1 - beta) * loss
        smooth_loss_corrected = smooth_loss / (1 - beta ** (i + 1))
        losses.append(smooth_loss_corrected)
        
        # 発散チェック
        if i > 0 and smooth_loss_corrected > 4 * losses[0]:
            break
        
        # 更新
        pos = pos - lr * grad
    
    return lrs[:len(losses)], losses


# テスト関数
def rosenbrock(x, y):
    return (1 - x)**2 + 100 * (y - x**2)**2

def rosenbrock_grad(x, y):
    dx = -2 * (1 - x) - 400 * x * (y - x**2)
    dy = 200 * (y - x**2)
    return np.array([dx, dy])


# LR Range Test
lrs, losses = lr_range_test(rosenbrock, rosenbrock_grad, start=(-1.0, 1.0), 
                            start_lr=1e-6, end_lr=1, num_iter=100)

# 可視化
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(lrs, losses, 'b-', linewidth=2)
ax.set_xscale('log')
ax.set_xlabel('学習率（対数スケール）')
ax.set_ylabel('損失')
ax.set_title('Learning Rate Range Test')
ax.grid(True, alpha=0.3)

# 推奨学習率の範囲をマーク
min_idx = np.argmin(losses)
suggested_lr = lrs[min_idx] / 10  # 最小点の1/10
ax.axvline(x=suggested_lr, color='red', linestyle='--', label=f'推奨: {suggested_lr:.2e}')
ax.legend()

plt.tight_layout()
plt.show()

print("【LR Range Testの使い方】")
print("1. 損失が最も急激に減少する領域を見つける")
print("2. その領域の1/10程度を初期学習率として設定")
print(f"3. この例では推奨学習率: {suggested_lr:.2e}")

### 3.2 ハイパーパラメータの優先順位

In [ ]:
print("="*80)
print("ハイパーパラメータチューニングの優先順位")
print("="*80)
print("")
print("【優先度: 高】")
print("-" * 40)
print("1. 学習率 (lr)")
print("   - 最も重要なパラメータ")
print("   - LR Range Testで探索")
print("")
print("2. バッチサイズ")
print("   - メモリが許す限り大きく")
print("   - ただし学習率の調整も必要")
print("")
print("【優先度: 中】")
print("-" * 40)
print("3. Weight Decay")
print("   - 1e-4 ~ 1e-2 の範囲で探索")
print("")
print("4. モーメンタム係数 (β1)")
print("   - 通常 0.9 で固定")
print("   - 特殊な場合のみ変更")
print("")
print("【優先度: 低】")
print("-" * 40)
print("5. Adam の β2")
print("   - 通常 0.999 で固定")
print("")
print("6. Adam の ε")
print("   - 通常 1e-8 で固定")
print("   - 精度問題がある場合のみ変更")

### 3.3 バッチサイズと学習率のスケーリング

In [ ]:
print("【バッチサイズと学習率のスケーリング則】")
print("")
print("バッチサイズを k 倍にしたとき：")
print("")
print("  1. Linear Scaling Rule:")
print("     lr_new = lr_base × k")
print("     例: batch 256→1024 (4倍) なら lr も4倍")
print("")
print("  2. Square Root Scaling:")
print("     lr_new = lr_base × √k")
print("     例: batch 256→1024 (4倍) なら lr は2倍")
print("")
print("-" * 50)
print("")
print("【どちらを使うか】")
print("")
print("- Linear Scaling: SGD + Warmup で一般的")
print("- Square Root: Adam で推奨されることも")
print("- 大バッチ (>1024): LARS/LAMB を使用")
print("")
print("【注意】")
print("- 学習率を大きくしたら必ず Warmup を使う")
print("- 線形スケーリングは無限には続かない")

# 可視化
batch_sizes = [32, 64, 128, 256, 512, 1024]
base_lr = 0.01
base_batch = 32

linear_lrs = [base_lr * (bs / base_batch) for bs in batch_sizes]
sqrt_lrs = [base_lr * np.sqrt(bs / base_batch) for bs in batch_sizes]

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(batch_sizes, linear_lrs, 'b-o', linewidth=2, label='Linear Scaling')
ax.plot(batch_sizes, sqrt_lrs, 'r-s', linewidth=2, label='Square Root Scaling')

ax.set_xlabel('バッチサイズ')
ax.set_ylabel('学習率')
ax.set_title('バッチサイズと学習率のスケーリング')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 4. 学習のデバッグと診断

In [ ]:
print("="*80)
print("学習曲線の診断")
print("="*80)
print("")
print("【パターン1: 損失が減少しない】")
print("-" * 40)
print("症状: 損失がほぼ一定")
print("原因: 学習率が小さすぎる / 勾配消失")
print("対策: ")
print("  - 学習率を10倍にしてみる")
print("  - 勾配のノルムを確認")
print("  - 初期化を確認")
print("")
print("【パターン2: 損失が振動/発散】")
print("-" * 40)
print("症状: 損失が大きく上下 / NaN")
print("原因: 学習率が大きすぎる")
print("対策: ")
print("  - 学習率を1/10にしてみる")
print("  - Gradient Clipping を追加")
print("  - Warmup を追加")
print("")
print("【パターン3: 訓練損失↓、検証損失↑】")
print("-" * 40)
print("症状: 過学習")
print("対策: ")
print("  - Weight Decay を増やす")
print("  - Dropout を追加/強化")
print("  - データ拡張を強化")
print("  - Early Stopping")
print("")
print("【パターン4: 両方の損失が高止まり】")
print("-" * 40)
print("症状: アンダーフィット")
print("対策: ")
print("  - モデルの容量を増やす")
print("  - 正則化を減らす")
print("  - 学習を長くする")

In [ ]:
# 学習曲線のパターンを可視化

np.random.seed(42)
epochs = np.arange(100)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# パターン1: 学習率が小さすぎる
train_loss_1 = 2.5 - 0.005 * epochs + np.random.randn(100) * 0.05
val_loss_1 = 2.5 - 0.004 * epochs + np.random.randn(100) * 0.08
axes[0, 0].plot(epochs, train_loss_1, 'b-', label='Train')
axes[0, 0].plot(epochs, val_loss_1, 'r-', label='Val')
axes[0, 0].set_title('パターン1: 学習率が小さすぎる')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# パターン2: 学習率が大きすぎる
train_loss_2 = 2.0 + 0.5 * np.sin(epochs * 0.5) + np.random.randn(100) * 0.3
axes[0, 1].plot(epochs, train_loss_2, 'b-')
axes[0, 1].set_title('パターン2: 学習率が大きすぎる（振動）')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].grid(True, alpha=0.3)

# パターン3: 過学習
train_loss_3 = 2.0 * np.exp(-0.05 * epochs) + 0.1 + np.random.randn(100) * 0.02
val_loss_3 = 2.0 * np.exp(-0.03 * epochs) + 0.3 + 0.01 * epochs + np.random.randn(100) * 0.05
axes[1, 0].plot(epochs, train_loss_3, 'b-', label='Train')
axes[1, 0].plot(epochs, val_loss_3, 'r-', label='Val')
axes[1, 0].set_title('パターン3: 過学習')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# パターン4: 良好な学習
train_loss_4 = 2.0 * np.exp(-0.04 * epochs) + 0.1 + np.random.randn(100) * 0.02
val_loss_4 = 2.0 * np.exp(-0.035 * epochs) + 0.15 + np.random.randn(100) * 0.03
axes[1, 1].plot(epochs, train_loss_4, 'b-', label='Train')
axes[1, 1].plot(epochs, val_loss_4, 'r-', label='Val')
axes[1, 1].set_title('良好な学習')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 5. 実践的なレシピ集

In [ ]:
print("="*80)
print("実践的なレシピ集")
print("="*80)
print("")
print("【レシピ1: ResNet on ImageNet】")
print("-" * 60)
print("""
optimizer = SGD(params, lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = MultiStepLR(optimizer, milestones=[30, 60, 90], gamma=0.1)

for epoch in range(100):
    train_one_epoch()
    scheduler.step()
""")

print("【レシピ2: BERT Fine-tuning】")
print("-" * 60)
print("""
optimizer = AdamW(params, lr=2e-5, weight_decay=0.01)

total_steps = len(dataloader) * num_epochs
warmup_steps = int(0.1 * total_steps)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

for batch in dataloader:
    loss.backward()
    torch.nn.utils.clip_grad_norm_(params, max_norm=1.0)
    optimizer.step()
    scheduler.step()
""")

print("【レシピ3: Vision Transformer (ViT)】")
print("-" * 60)
print("""
optimizer = AdamW(params, lr=1e-3, betas=(0.9, 0.999), weight_decay=0.3)

scheduler = CosineAnnealingLR(optimizer, T_max=300, eta_min=1e-5)
warmup_scheduler = LinearWarmup(optimizer, warmup_epochs=5)

for epoch in range(300):
    if epoch < 5:
        warmup_scheduler.step()
    else:
        scheduler.step()
""")

print("【レシピ4: Fast.ai スタイル (One Cycle)】")
print("-" * 60)
print("""
# LR Range Test で max_lr を決定
max_lr = 1e-2

optimizer = Adam(params, lr=max_lr/25)  # 初期LRは max_lr/div_factor
scheduler = OneCycleLR(
    optimizer,
    max_lr=max_lr,
    total_steps=len(dataloader) * num_epochs,
    pct_start=0.3,
    div_factor=25,
    final_div_factor=1e4
)

for batch in dataloader:
    loss.backward()
    optimizer.step()
    scheduler.step()
""")

---

## 6. よくある問題と解決策

In [ ]:
print("="*80)
print("よくある問題と解決策")
print("="*80)
print("")

print("【問題1: NaN が発生する】")
print("-" * 60)
print("チェックリスト:")
print("  □ 学習率が大きすぎないか")
print("  □ データに inf/nan が含まれていないか")
print("  □ 損失関数で log(0) が発生していないか")
print("  □ 勾配爆発が起きていないか")
print("解決策:")
print("  - Gradient Clipping を追加")
print("  - 学習率を下げる")
print("  - 混合精度の場合は Loss Scaling を確認")
print("")

print("【問題2: 損失が一定で減少しない】")
print("-" * 60)
print("チェックリスト:")
print("  □ モデルの勾配が流れているか")
print("  □ データローダーが正しく動作しているか")
print("  □ 損失関数の実装が正しいか")
print("解決策:")
print("  - 1バッチだけで過学習できるか確認")
print("  - 勾配のノルムをログに出力")
print("  - 学習率を上げてみる")
print("")

print("【問題3: 訓練精度は高いが検証精度が低い】")
print("-" * 60)
print("チェックリスト:")
print("  □ データ拡張は適切か")
print("  □ 正則化（Dropout, Weight Decay）は十分か")
print("  □ 訓練/検証データの分布は同じか")
print("解決策:")
print("  - Weight Decay を増やす（0.01 → 0.1）")
print("  - Dropout を追加/強化")
print("  - データ拡張を強化")
print("  - モデルを小さくする")
print("")

print("【問題4: 学習が遅い】")
print("-" * 60)
print("チェックリスト:")
print("  □ 学習率は最適か")
print("  □ バッチサイズは適切か")
print("  □ データローダーがボトルネックでないか")
print("解決策:")
print("  - LR Range Test で最適な学習率を探す")
print("  - バッチサイズを大きくする")
print("  - num_workers を増やす")
print("  - 混合精度学習を使用する")

---

## 7. Phase 10 総まとめ

In [ ]:
print("="*80)
print("Phase 10: Optimization Methods 総まとめ")
print("="*80)
print("")
print("【学んだ内容】")
print("")
print("Notebook 110: 最適化の基礎と勾配降下法")
print("  - 最適化問題の定式化")
print("  - 勾配降下法のアルゴリズム")
print("  - 学習率の影響")
print("  - バッチサイズの選択")
print("")
print("Notebook 111: Momentum と Nesterov")
print("  - SGDの問題点（振動）")
print("  - Momentum による改善")
print("  - Nesterov の先読み")
print("")
print("Notebook 112: 適応学習率手法")
print("  - Adagrad の仕組みと問題点")
print("  - RMSprop による改善")
print("  - Adam の完全な理解")
print("  - AdamW と Weight Decay")
print("")
print("Notebook 113: 学習率スケジューリング")
print("  - Step Decay, Exponential Decay")
print("  - Cosine Annealing")
print("  - Warmup の重要性")
print("  - One Cycle Policy")
print("")
print("Notebook 114: 正則化と最適化")
print("  - L1/L2 正則化")
print("  - Weight Decay vs L2")
print("  - Dropout")
print("  - Batch Normalization")
print("")
print("Notebook 115: 高度な最適化テクニック")
print("  - Gradient Clipping")
print("  - SAM")
print("  - LARS/LAMB")
print("  - Lookahead")
print("")
print("Notebook 116: オプティマイザの選び方と実践")
print("  - 選択フローチャート")
print("  - タスク別推奨設定")
print("  - デバッグと診断")
print("")

In [ ]:
print("="*80)
print("クイックリファレンス: オプティマイザ選択表")
print("="*80)
print("")
print(f"{'タスク':<25} {'オプティマイザ':<15} {'学習率':<12} {'備考'}")
print("-"*80)
print(f"{'画像分類 (CNN)':<25} {'SGD+Momentum':<15} {'0.1':<12} {'Step Decay'}")
print(f"{'NLP (Transformer)':<25} {'AdamW':<15} {'1e-4~5e-4':<12} {'Warmup必須'}")
print(f"{'ファインチューニング':<25} {'AdamW':<15} {'1e-5~3e-5':<12} {'元の1/10'}")
print(f"{'GAN':<25} {'Adam':<15} {'2e-4':<12} {'β1=0.5'}")
print(f"{'強化学習':<25} {'Adam':<15} {'3e-4':<12} {'Grad Clip'}")
print(f"{'大バッチ (>1K) CNN':<25} {'LARS':<15} {'スケール':<12} {'Warmup必須'}")
print(f"{'大バッチ Transformer':<25} {'LAMB':<15} {'スケール':<12} {'Warmup必須'}")
print(f"{'汎化重視':<25} {'SAM+base':<15} {'base依存':<12} {'2x計算コスト'}")
print("-"*80)
print("")
print("【迷ったときのデフォルト】")
print("")
print("optimizer = AdamW(params, lr=1e-3, weight_decay=0.01)")
print("scheduler = CosineAnnealingLR(optimizer, T_max=epochs)")

---

## 参考文献

### 基本文献
1. Ruder, S. (2016). An overview of gradient descent optimization algorithms. *arXiv:1609.04747*.
2. Kingma, D. P., & Ba, J. (2015). Adam: A method for stochastic optimization. *ICLR*.

### 発展文献
3. Loshchilov, I., & Hutter, F. (2019). Decoupled weight decay regularization. *ICLR*.
4. Smith, L. N. (2017). Cyclical learning rates for training neural networks. *WACV*.
5. You, Y., et al. (2020). Large batch optimization for deep learning: Training BERT in 76 minutes. *ICLR*.
6. Foret, P., et al. (2021). Sharpness-aware minimization for efficiently improving generalization. *ICLR*.

### 実践ガイド
7. PyTorch Recipes: https://pytorch.org/tutorials/recipes/recipes_index.html
8. Fast.ai Course: https://www.fast.ai/